## Import library

In [1]:
import lightgbm as lgb

from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc,roc_curve,log_loss

import numpy as np
import pandas as pd
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial
import gc

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel

from tqdm import tqdm_notebook


pd.options.display.max_columns = 200

## Data import

In [8]:
train_gal = pd.read_pickle('../input/train_gal_20181214.pickle')
train_ex_gal = pd.read_pickle('../input/train_ex_gal_20181214.pickle')
test_gal = pd.read_pickle('../input/test_gal_20181214.pickle')
test_ex_gal = pd.read_pickle('../input/test_ex_gal_20181214.pickle')

In [9]:
train_sixth_band = pd.read_pickle('../input/train_sixth_byband.pickle')
test_sixth_band = pd.read_pickle('../input/test_sixth_byband.pickle')

In [10]:
train_fix_band_0_stats = pd.read_pickle('../input/train_band_0_fix_stats.pickle')
test_fix_band_0_stats = pd.read_pickle('../input/test_band_0_fix_stats.pickle')
train_fix_band_1_stats = pd.read_pickle('../input/train_band_1_fix_stats.pickle')
test_fix_band_1_stats = pd.read_pickle('../input/test_band_1_fix_stats.pickle')

In [11]:
train_gal = pd.merge(train_gal,train_fix_band_0_stats,on='object_id',how='left')
train_gal = pd.merge(train_gal,train_fix_band_1_stats,on='object_id',how='left')

train_ex_gal = pd.merge(train_ex_gal,train_sixth_band,on='object_id',how='left')
train_ex_gal = pd.merge(train_ex_gal,train_fix_band_0_stats,on='object_id',how='left')
train_ex_gal = pd.merge(train_ex_gal,train_fix_band_1_stats,on='object_id',how='left')



In [12]:
test_gal = pd.merge(test_gal,test_fix_band_0_stats,on='object_id',how='left')
test_gal = pd.merge(test_gal,test_fix_band_1_stats,on='object_id',how='left')

test_ex_gal = pd.merge(test_ex_gal,test_sixth_band,on='object_id',how='left')
test_ex_gal = pd.merge(test_ex_gal,test_fix_band_0_stats ,on='object_id',how='left')
test_ex_gal = pd.merge(test_ex_gal,test_fix_band_1_stats ,on='object_id',how='left')

In [13]:
train_ex_gal.head(3)

,index,object_id,gal_b,hostgal_photoz,hostgal_photoz_err,target,haversine,latlon1,diff_hostgal,diff_hostgal_abs,passband_0_ratio_detect,passband_1_ratio_detect,passband_2_ratio_detect,passband_3_ratio_detect,passband_4_ratio_detect,passband_5_ratio_detect,passband_0_ratio,passband_1_ratio,passband_2_ratio,passband_3_ratio,passband_4_ratio,passband_5_ratio,ratio_by_band_median,ratio_by_band_var,flux_max_detected,flux_min_detected,flux_mean_detected,flux_median_detected,flux_skew_detected,flux_max_min_detected,flux_max,flux_min,flux_mean,flux_median,flux_skew,flux_max_min,detected_mean,mjd_diff,mjd_halfflux,mjd_sixthflux,diff_baba_mjd,max_flux_0_1,"flux__fft_coefficient__coeff_0__attr_""abs""","flux__fft_coefficient__coeff_1__attr_""abs""",flux__longest_strike_above_mean,flux__number_crossing_m__m_1,symm,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__longest_strike_above_mean,0__number_crossing_m__m_1,0__ratio_beyond_r_sigma__r_0.5,0__sample_entropy,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__longest_strike_above_mean,1__number_crossing_m__m_1,1__ratio_beyond_r_sigma__r_0.5,1__sample_entropy,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__longest_strike_above_mean,2__number_crossing_m__m_1,2__ratio_beyond_r_sigma__r_0.5,2__sample_entropy,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__longest_strike_above_mean,3__number_crossing_m__m_1,3__ratio_beyond_r_sigma__r_0.5,3__sample_entropy,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__longest_strike_above_mean,4__number_crossing_m__m_1,4__ratio_beyond_r_sigma__r_0.5,4__sample_entropy,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__longest_strike_above_mean,5__number_crossing_m__m_1,5__ratio_beyond_r_sigma__r_0.5,5__sample_entropy,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_dif,flux_dif2,flux_w_mean,flux_dif3,"flux_max__fft_coefficient__coeff_0__attr_""abs""","flux_max__fft_coefficient__coeff_1__attr_""abs""",flux_max__kurtosis,flux_max__skewness,"flux_min__fft_coefficient__coeff_0__attr_""abs""","flux_min__fft_coefficient__coeff_1__attr_""abs""",flux_min__kurtosis,flux_min__skewness,"flux_mean__fft_coefficient__coeff_0__attr_""abs""",...,flux_err_rate_min,flux_err_rate_mean,flux_err_rate_median,flux_err_rate_var,flux_err_rate_skew,flux_err_rate_max_min,0__kurtosis,0__skewness,1__kurtosis,1__skewness,2__kurtosis,2__skewness,3__kurtosis,3__skewness,4__kurtosis,4__skewness,5__kurtosis,5__skewness,0__maximum,0__mean,0__median,0__minimum,0__standard_deviation,1__maximum,1__mean,1__median,1__minimum,1__standard_deviation,2__maximum,2__mean,2__median,2__minimum,2__standard_deviation,3__maximum,3__mean,3__median,3__minimum,3__standard_deviation,4__maximum,4__mean,4__median,4__minimum,4__standard_deviation,5__maximum,5__mean,5__median,5__minimum,5__standard_deviation,diff_flux__kurtosis_diff,diff_flux__maximum_diff,diff_flux__mean_diff,diff_flux__median_diff,diff_flux__minimum_diff,diff_flux__skewness_diff,diff_flux__standard_deviation_diff,diff_flux__maximum_diff_det,diff_flux__median_diff_det,diff_flux__minimum_diff_det,0__maximum_diff,0__median_diff,0__minimum_diff,1__maximum_diff,1__median_diff,1__minimum_diff,2__maximum_diff,2__median_diff,2__minimum_diff,3__maximum_diff,3__median_diff,3__minimum_diff,4__maximum_diff,4__median_diff,4__minimum_diff,5__maximum_diff,5__median_diff,5__minimum_diff,sixth_band_0,sixth_band_1,sixth_band_2,sixth_band_3,sixth_band_4,sixth_band_5,second_band_0,second_band_1,second_band_2,second_band_3,second_band_4,second_band_5,band_0_flux_max,band_0_flux_min,band_0_flux_mean,band_0_flux_med,band_0_flux_skew,band_0_flux_std,band_1_flux_max,band_1_flux_min,band_1_flux_mean,band_1_flux_med,band_1_flux_skew,band_1_flux_std
0,0,615,-51.753706,0.0000,0.0000,9

In [14]:
train_ex_gal['max_fix_flux_diff_0_1'] = train_ex_gal['band_0_flux_max'] - train_ex_gal['band_1_flux_max']

In [15]:
test_ex_gal['max_fix_flux_diff_0_1'] = test_ex_gal['band_0_flux_max'] - test_ex_gal['band_1_flux_max']

In [16]:
def check_null(data):
    print(data.isnull().sum())

pd.set_option('display.max_rows', 500)
#print(len(train_extra_galaxy))
#print(len(test_extra_galaxy))
#print(check_null(train_gal))
#print(check_null(train_oof_summary))
#print(check_null(test_oof_summary))
#print(check_null(train_ex_gal))
#print(check_null(test_ex_gal))
#print(check_null(test_gal))
#print(check_null(test_extra_galaxy))

In [17]:
#drop more feature
drop_list = ['0__minimum_diff','1__minimum_diff','2__minimum_diff','3__minimum_diff','4__minimum_diff','5__minimum_diff'
             ,'diff_flux__minimum_diff','diff_flux__minimum_diff_det','index','flux_dif']

In [18]:
train_ex_gal = train_ex_gal.drop(drop_list,axis=1)
test_ex_gal = test_ex_gal.drop(drop_list,axis=1)

In [19]:
train_gal = train_gal.replace([np.inf, -np.inf], np.nan)
train_ex_gal = train_ex_gal.replace([np.inf, -np.inf], np.nan)

train_gal_mean = train_gal.mean(axis=0)
train_ex_gal_mean = train_ex_gal.mean(axis=0)

train_gal.fillna(train_gal_mean, inplace=True)
train_ex_gal.fillna(train_ex_gal_mean, inplace=True)

In [20]:
# add mean → good result

test_gal = test_gal.replace([np.inf, -np.inf], np.nan)
test_ex_gal = test_ex_gal.replace([np.inf, -np.inf], np.nan)

test_gal = test_gal.fillna(train_gal_mean)
test_ex_gal = test_ex_gal.fillna(train_ex_gal_mean)

In [21]:
import sys
# check used_momery
print(train_gal.shape,"行×列")
print("train_gal:使用メモリ",round(sys.getsizeof(train_gal) / 1073741824,2),"GB")

print(train_ex_gal.shape,"行×列")
print("train_gal:使用メモリ",round(sys.getsizeof(train_ex_gal) / 1073741824,2),"GB")

(7848, 167) 行×列
train_gal:使用メモリ 0.01 GB
(7848, 210) 行×列
train_gal:使用メモリ 0.01 GB


In [22]:
import sys
# check used_momery
print(test_gal.shape,"行×列")
print("test:使用メモリ",round(sys.getsizeof(test_gal) / 1073741824,2),"GB")

print(test_ex_gal.shape,"行×列")
print("test:使用メモリ",round(sys.getsizeof(test_ex_gal) / 1073741824,2),"GB")

(3492890, 166) 行×列
test:使用メモリ 3.92 GB
(3492890, 209) 行×列
test:使用メモリ 4.89 GB


## modeling_lgbm

In [23]:
# split galaxica
train_galaxy = train_gal[train_gal['hostgal_photoz'] == 0]
train_extra_galaxy = train_ex_gal[train_ex_gal['hostgal_photoz'] != 0]

In [24]:
test_galaxy = test_gal[test_gal['hostgal_photoz'] == 0]
test_extra_galaxy = test_ex_gal[test_ex_gal['hostgal_photoz'] != 0]

In [25]:
gal_y = train_galaxy['target']
ex_gal_y = train_extra_galaxy['target']
train_gal_id = train_galaxy['object_id']
train_ex_gal_id = train_extra_galaxy['object_id']

del train_galaxy['target'],train_extra_galaxy['target']

In [26]:
del train_extra_galaxy['object_id'],train_galaxy['object_id']

In [27]:
test_gal_id = test_galaxy['object_id']
test_ex_gal_id = test_extra_galaxy['object_id']
del test_galaxy['object_id']

In [28]:
del test_extra_galaxy['object_id']

In [29]:
# binary val_galaxy
from sklearn.preprocessing import OneHotEncoder
target_enc = OneHotEncoder()
gal_X = target_enc.fit_transform(gal_y.values.reshape(-1,1)).toarray()
ex_gal_X = target_enc.fit_transform(ex_gal_y.values.reshape(-1,1)).toarray()
gal_class_binary = pd.DataFrame(gal_X, columns = ["target_6","target_16","target_53","target_65","target_92"])
ex_gal_class_binary = pd.DataFrame(ex_gal_X, columns = ["target_15","target_42","target_52","target_62","target_64","target_67","target_88","target_90","target_95"])

In [49]:
def train_binary_classifiers(full_train=None, y=None, seed=1):

    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    clfs = []
    logloss = []
    importances = pd.DataFrame()
    # conclusion:defalut is great!
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        #'num_class': 14,
        'metric': 'logloss', #1214 change logloss
        'learning_rate': 0.005, #def:0.03
        'subsample': .8, #def:.8
        'colsample_bytree': .7, #def:.7
        'reg_alpha': .01, #def:.01 
        'reg_lambda': .01, #def:.01 
        'min_split_gain': 0.01, #def:0.01
        'min_child_weight': 10, # def:10
        'n_estimators': 100000,
        'silent': -1,
        'verbose': -1,
        'max_depth': 7,
        #'sample_pos_weight':pos_weight
    }
    
    oof_preds = np.zeros((len(full_train), np.unique(y).shape[0]))
    
    # photoz_split
    target_photoz = pd.cut(full_train.hostgal_photoz, 10)
    le = preprocessing.LabelEncoder()
    le.fit(target_photoz)
    target_photoz = le.transform(target_photoz) 
    
    #for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    for fold_, (trn_, val_) in enumerate(folds.split(target_photoz, target_photoz)):    
        trn_x, trn_y = full_train.iloc[trn_], y.iloc[trn_]
        val_x, val_y = full_train.iloc[val_], y.iloc[val_]

        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            #eval_metric=lgb_multi_weighted_logloss,
            eval_metric='binary_logloss',
            verbose=2000,
            early_stopping_rounds=100,
            #sample_weight=trn_y.map(weights)
            #callbacks=[lgb.reset_parameter(max_depth=[4] * 100 + [2] * 300 + [1] * 3600)]
            #callbacks=[lgb.reset_parameter(
            #    learning_rate=[0.01] * 100 + [0.03] * 1000 + [0.05] * 2900
            #    ,max_depth=[5] * 100 + [3] * 1000 + [2] * 2900
            #)]
        )
        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        #get_logger().info(multi_weighted_logloss(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)))

        imp_df = pd.DataFrame()
        imp_df['feature'] = full_train.columns
        imp_df['gain'] = clf.feature_importances_
        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        clfs.append(clf)
    
    logloss = log_loss(y, oof_preds)
    #fpr, tpr, thresholds = roc_curve(y, oof_preds[:,1], pos_label=1)
    print('total_logloss : %.5f ' % logloss)
    #auc_class = auc(fpr, tpr)
    
    return clfs, importances, oof_preds, logloss


In [45]:
import matplotlib.pyplot as plt
import seaborn as sns
def save_importances(importances_,layout=True):
    mean_gain = importances_[['gain', 'feature']].groupby('feature').mean()
    importances_['mean_gain'] = importances_['feature'].map(mean_gain['gain'])
    if layout==True:
        plt.figure(figsize=(8, 50))
        sns.barplot(x='gain', y='feature', data=importances_.sort_values('mean_gain', ascending=False))
        plt.tight_layout()
    else:
        None

In [46]:
def galaxy_model(galaxy='gal',pred=False, layout=False, seed_list=[1,2]):
    importance_dm = pd.DataFrame()
    oof_merge = pd.DataFrame()
    preds_merge = pd.DataFrame()
    oof_total = pd.DataFrame()
    preds_total = pd.DataFrame()
    logloss_total = []
    
    lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    #'num_class': 14,
    'metric': 'logloss', #1214 change logloss
    'learning_rate': 0.03, #def:0.03
    'subsample': .8, #def:.8
    'colsample_bytree': .7, #def:.7
    'reg_alpha': .01, #def:.01 
    'reg_lambda': .01, #def:.01 
    'min_split_gain': 0.01, #def:0.01
    'min_child_weight': 10, # def:10
    'n_estimators': 100000,
    'silent': -1,
    'verbose': -1,
    'max_depth': 3,
    }
    clf = lgb.LGBMClassifier(**lgb_params)
    
    # seed averaging
    for seed_num in seed_list:
        print('------------ seed {} start ------------'.format(seed_num))
        
        if galaxy=='gal':
            for i in range(gal_class_binary.shape[1]):

                y = gal_class_binary.iloc[:,i]
                
                clfs, importances, oof_preds, logloss = train_binary_classifiers(train_galaxy, y, seed_num)
                oof_preds = pd.DataFrame(oof_preds[:,1])
                save_importances(importances_=importances,layout=layout)
                importances = importances[['feature','mean_gain']].drop_duplicates()
                importances.columns = ['feature',y.to_frame().columns[0]]

                if pred == True:
                    preds_ = None
                    for clf in tqdm_notebook(clfs):
                        if preds_ is None:
                            preds_ = clf.predict_proba(test_galaxy) / len(clfs)
                        else:
                            preds_ += clf.predict_proba(test_galaxy) / len(clfs)
                    preds_ = pd.DataFrame(preds_[:,1])

                if i == 0:
                    importance_dm = importances
                    oof_total = oof_preds

                    if pred == True:
                        preds_total = preds_
                else:
                    importance_dm = pd.merge(importance_dm,importances,on='feature',how='left')
                    oof_total = pd.merge(oof_total,oof_preds,left_index=True,right_index=True) 
                    if pred == True:
                        preds_total = pd.merge(preds_total,preds_,left_index=True,right_index=True)
                logloss_total.append(logloss)

            oof_total.columns = ["target_6","target_16","target_53","target_65","target_92"]

            if pred == True:
                preds_total.columns = ["target_6","target_16","target_53","target_65","target_92"]

        if galaxy=='ex_gal':        
            for i in range(ex_gal_class_binary.shape[1]):
                y = ex_gal_class_binary.iloc[:,i]
                
                #selector
                #lgb_selector = SelectFromModel(clf)
                #lgb_selector.fit(train_extra_galaxy, y)
                #feature_idx = lgb_selector.get_support() 
                #feature_name = train_extra_galaxy.columns[feature_idx]

                clfs, importances, oof_preds, logloss = train_binary_classifiers(train_extra_galaxy, y, seed_num)

                oof_preds = pd.DataFrame(oof_preds[:,1])
                save_importances(importances_=importances,layout=layout)
                importances = importances[['feature','mean_gain']].drop_duplicates()
                importances.columns = ['feature',y.to_frame().columns[0]]

                if pred == True:
                    preds_ = None

                    for clf in tqdm_notebook(clfs):
                        if preds_ is None:
                            preds_ = clf.predict_proba(test_extra_galaxy) / len(clfs)
                        else:
                            preds_ += clf.predict_proba(test_extra_galaxy) / len(clfs)

                    preds_ = pd.DataFrame(preds_[:,1])

                if i == 0:
                    importance_dm = importances
                    oof_total = oof_preds
                    if pred == True:
                        preds_total = preds_
                else:
                    importance_dm = pd.merge(importance_dm,importances,on='feature',how='left')
                    oof_total = pd.merge(oof_total,oof_preds,left_index=True,right_index=True) 
                    if pred == True:
                        preds_total = pd.merge(preds_total,preds_,left_index=True,right_index=True)
                logloss_total.append(logloss)

            oof_total.columns = ["target_15","target_42","target_52","target_62","target_64","target_67","target_88","target_90","target_95"]
            if pred == True:
                preds_total.columns = ["target_15","target_42","target_52","target_62","target_64","target_67","target_88","target_90","target_95"]
    
        # merge seed_result
        if seed_num == 1:
            oof_merge = oof_total / len(seed_list)
            if pred == True:
                preds_merge = preds_total / len(seed_list)
        else:
            oof_merge += oof_total / len(seed_list)
            if pred == True:
                preds_merge += preds_total / len(seed_list)

    if pred == True:    
        #return importance_dm,oof_total,preds_total,logloss_total
        return importance_dm,oof_merge,preds_merge,logloss_total
    else:
        #return importance_dm,oof_total,logloss_total
        return importance_dm,oof_merge,logloss_total

In [280]:
%%time
# 予測
print('---------------  start gal_model pred  ---------------')
importances_gal,oof_gal_total,preds_gal,logloss_gal = galaxy_model(galaxy='gal',pred=True,layout=False,seed_list=[1])

print('---------------  start ex_gal_model pred  ---------------')
importances_ex_gal,oof_ex_gal_total,preds_ex_gal,logloss_ex_gal = galaxy_model(galaxy='ex_gal',pred=True,layout=False,seed_list=[1])

print('---------------  done model pred  ---------------')
print(np.round(logloss_gal, decimals=4))
print('total_gal_logloss:',np.array(logloss_gal).mean())
print(np.round(logloss_ex_gal, decimals=4))
print('total_ex_gal_logloss:',np.array(logloss_ex_gal).mean())

---------------  start gal_model pred  ---------------
------------ seed 1 start ------------
Training until validation scores don't improve for 50 rounds.
[1000]	training's binary_logloss: 0.0146161	valid_1's binary_logloss: 0.0351148
Early stopping, best iteration is:
[961]	training's binary_logloss: 0.0146161	valid_1's binary_logloss: 0.0351148
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[846]	training's binary_logloss: 0.0144954	valid_1's binary_logloss: 0.0354625
Training until validation scores don't improve for 50 rounds.
[1000]	training's binary_logloss: 0.0146415	valid_1's binary_logloss: 0.0280187
Early stopping, best iteration is:
[978]	training's binary_logloss: 0.0146415	valid_1's binary_logloss: 0.0280187
Training until validation scores don't improve for 50 rounds.
[1000]	training's binary_logloss: 0.0157181	valid_1's binary_logloss: 0.0349502
Early stopping, best iteration is:
[962]	training's binary_logloss: 0.015718


Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[630]	training's binary_logloss: 0.0139718	valid_1's binary_logloss: 0.039746
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[641]	training's binary_logloss: 0.0134806	valid_1's binary_logloss: 0.0327738
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[630]	training's binary_logloss: 0.0132439	valid_1's binary_logloss: 0.0364806
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[705]	training's binary_logloss: 0.0134289	valid_1's binary_logloss: 0.0236402
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[669]	training's binary_logloss: 0.0130456	valid_1's binary_logloss: 0.0440345
total_logloss : 0.03534 



Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[93]	training's binary_logloss: 0.0186759	valid_1's binary_logloss: 0.0171776
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[107]	training's binary_logloss: 0.0197236	valid_1's binary_logloss: 0.0182755
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[98]	training's binary_logloss: 0.0255409	valid_1's binary_logloss: 0.0343083
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[103]	training's binary_logloss: 0.0224742	valid_1's binary_logloss: 0.0267565
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[101]	training's binary_logloss: 0.0225492	valid_1's binary_logloss: 0.0267664
total_logloss : 0.02466 



Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[653]	training's binary_logloss: 0.0128052	valid_1's binary_logloss: 0.0228059
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[618]	training's binary_logloss: 0.0129591	valid_1's binary_logloss: 0.0382382
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[662]	training's binary_logloss: 0.0128362	valid_1's binary_logloss: 0.0192374
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[673]	training's binary_logloss: 0.0133222	valid_1's binary_logloss: 0.0252627
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[621]	training's binary_logloss: 0.0131557	valid_1's binary_logloss: 0.0511184
total_logloss : 0.03133 



Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[226]	training's binary_logloss: 0.0173008	valid_1's binary_logloss: 0.0285877
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[236]	training's binary_logloss: 0.0172757	valid_1's binary_logloss: 0.0249926
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[220]	training's binary_logloss: 0.0174957	valid_1's binary_logloss: 0.030339
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[218]	training's binary_logloss: 0.0176739	valid_1's binary_logloss: 0.0387089
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[225]	training's binary_logloss: 0.0171905	valid_1's binary_logloss: 0.03058
total_logloss : 0.03064 


...


Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[277]	training's binary_logloss: 0.0408441	valid_1's binary_logloss: 0.0999234
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[273]	training's binary_logloss: 0.040561	valid_1's binary_logloss: 0.0933843
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[335]	training's binary_logloss: 0.03474	valid_1's binary_logloss: 0.0975569
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[251]	training's binary_logloss: 0.0410506	valid_1's binary_logloss: 0.114267
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[285]	training's binary_logloss: 0.0388552	valid_1's binary_logloss: 0.108673
total_logloss : 0.10274 



Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[184]	training's binary_logloss: 0.00655337	valid_1's binary_logloss: 0.0174031
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[199]	training's binary_logloss: 0.00660881	valid_1's binary_logloss: 0.00439414
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[159]	training's binary_logloss: 0.00678619	valid_1's binary_logloss: 0.0234121
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[197]	training's binary_logloss: 0.00663707	valid_1's binary_logloss: 0.00528497
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[181]	training's binary_logloss: 0.00709975	valid_1's binary_logloss: 0.0114123
total_logloss : 0.01239 



Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[872]	training's binary_logloss: 0.0772249	valid_1's binary_logloss: 0.286534
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[854]	training's binary_logloss: 0.0780515	valid_1's binary_logloss: 0.328921
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[794]	training's binary_logloss: 0.0885213	valid_1's binary_logloss: 0.298572
Training until validation scores don't improve for 50 rounds.
[1000]	training's binary_logloss: 0.0684267	valid_1's binary_logloss: 0.25339
Early stopping, best iteration is:
[1409]	training's binary_logloss: 0.0437542	valid_1's binary_logloss: 0.247528
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[838]	training's binary_logloss: 0.0786419	valid_1's binary_logloss: 0.280389
total_logloss : 0.28843 



Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[263]	training's binary_logloss: 0.0138467	valid_1's binary_logloss: 0.0297861
Training until validation scores don't improve for 50 rounds.
[1000]	training's binary_logloss: 0.0084041	valid_1's binary_logloss: 0.0228241
Early stopping, best iteration is:
[1423]	training's binary_logloss: 0.00733938	valid_1's binary_logloss: 0.0223329
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[368]	training's binary_logloss: 0.0113907	valid_1's binary_logloss: 0.0376164
Training until validation scores don't improve for 50 rounds.
[1000]	training's binary_logloss: 0.00828575	valid_1's binary_logloss: 0.0183805
Early stopping, best iteration is:
[1409]	training's binary_logloss: 0.00725254	valid_1's binary_logloss: 0.0179664
Training until validation scores don't improve for 50 rounds.
[1000]	training's binary_logloss: 0.0081361	valid_1's binary_logloss


---------------  done model pred  ---------------
[0.0378 0.0353 0.0247 0.0313 0.0306]
total_gal_logloss: 0.031957967311322784
[0.1203 0.3141 0.1176 0.1969 0.0183 0.1027 0.0124 0.2884 0.0263]
total_ex_gal_logloss: 0.13298510203787375
CPU times: user 1h 25min 30s, sys: 19min 24s, total: 1h 44min 54s
Wall time: 28min 52s


In [51]:
# check shape
print(oof_gal_total.shape)
print(oof_ex_gal_total.shape)
print(preds_gal.shape)
print(preds_ex_gal.shape)

(2325, 5)
(5523, 9)
(390510, 5)
(3102380, 9)


In [52]:
oof_gal_total['prob_sum'] = oof_gal_total.iloc[:,1:].sum(axis=1)
oof_gal_total['prob_med'] = oof_gal_total.iloc[:,1:].median(axis=1)
oof_ex_gal_total['prob_sum'] = oof_ex_gal_total.iloc[:,1:].sum(axis=1)
oof_ex_gal_total['prob_med'] = oof_ex_gal_total.iloc[:,1:].median(axis=1)

preds_gal['prob_sum'] = preds_gal.iloc[:,1:].sum(axis=1)
preds_gal['prob_med'] = preds_gal.iloc[:,1:].median(axis=1)
preds_ex_gal['prob_sum'] = preds_ex_gal.iloc[:,1:].sum(axis=1)
preds_ex_gal['prob_med'] = preds_ex_gal.iloc[:,1:].median(axis=1)

In [282]:
#importance_ex_gal,oof_ex_gal_total,preds_ex_gal,auc_class_exgal = galaxy_model(galaxy='ex_gal',pred=True)

In [53]:
train_ex_gal_id = train_ex_gal_id.reset_index()
train_gal_id = train_gal_id.reset_index()
test_ex_gal_id = test_ex_gal_id.reset_index()
test_gal_id = test_gal_id.reset_index()
del train_ex_gal_id['index'],train_gal_id['index'],test_ex_gal_id['index'],test_gal_id['index']

In [54]:
oof_gal = pd.merge(train_gal_id,oof_gal_total,left_index=True,right_index=True,how='left')
preds_gal = pd.merge(test_gal_id,preds_gal,left_index=True,right_index=True,how='left')
oof_ex_gal = pd.merge(train_ex_gal_id,oof_ex_gal_total,left_index=True,right_index=True,how='left')
preds_ex_gal = pd.merge(test_ex_gal_id,preds_ex_gal,left_index=True,right_index=True,how='left')

In [55]:
oof_result = oof_gal.append(oof_ex_gal,sort=True).fillna(0)
preds_result = preds_gal.append(preds_ex_gal,sort=True).fillna(0)

In [56]:
print(preds_result.shape)
print(oof_result.shape)

(3492890, 17)
(7848, 17)


In [57]:
preds_result.head(3)

,object_id,prob_med,prob_sum,target_15,target_16,target_42,target_52,target_53,target_6,target_62,target_64,target_65,target_67,target_88,target_90,target_92,target_95
0,168,0.004919,1.004749,0.0,0.998177,0.0,0.0,0.000801,0.000310,0.0,0.0,0.004919,0.0,0.0,0.0,0.000853,0.0
1,1063,0.025563,1.011062,0.0,0.000449,0.0,0.0,0.025563,0.006437,0.0,0.0,0.002186,0.0,0.0,0.0,0.982864,0.0
2,2270,0.001267,0.998139,0.0,0.000955,0.0,0.0,0.001128,0.004003,0.0,0.0,0.994789,0.0,0.0,0.0,0.001267,0.0


In [58]:
oof_result.to_pickle('../input/oof_result_20181217_v2.pickle')
preds_result.to_pickle('../input/preds_result_20181217_v2.pickle')